In [1]:
import numpy as np
import pandas as pd
import os
from itertools import permutations

In [2]:
data_path = "dane/"
df = pd.read_excel(os.path.join(data_path, "dane1.xlsx"))

In [3]:
class Individual:
    def __init__(self, order):
        self.order = order
        self.value = self.__calculate_value_of_order()
        
    def __repr__(self):
        return str(self.value)
    
    def __print__(self):
        print("Order")
        print(self.order)
        print(f"Value of order: {self.value}")

    def __calculate_value_of_order(self):
        df = self.order
        if len(df) <= 0:
            return 0
        elif len(df) == 1:
            return int(df.sum(axis=1))

        obrobione = df.copy()
        obrobione["M1"] = df["M1"].cumsum() # TODO
        obrobione.iloc[1] = df.iloc[1].cumsum()

        for i in range(2, len(obrobione)):
            for j in range(2, len(obrobione.iloc[1])):
                max_val = max(int(obrobione.iloc[[i-1], [j]].values),
                                 int(obrobione.iloc[[i], [j-1]].values))
                obrobione.iloc[[i], [j]] = max_val + df.iloc[[i], [j]]
        wartosc = int(obrobione.iloc[[-1], [-1]].values)
        return wartosc
    
    def cross(self, other, cross_type = 1):
        if cross_type == 1:
            return self.__crossing_type_1(other)
        elif cross_type == 2:
            return self.__crossing_type_2(other)
    
    def __crossing_type_1(self, other):
        """
        Pierwszy sposób krzyżowania - OX crossing
            Krótki opis.
        """ 
        l = len(self.order)
        border = l//5
        left_b, right_b = np.random.randint(border, l//2), np.random.randint(l//2, l - border)

        child_1 = [None for i in range(l)]
        child_2 = [None for i in range(l)]

        for i in range(left_b, right_b):
            child_1[i] = self.order["Zadanie"].iloc[i]
            child_2[i] = other.order["Zadanie"].iloc[i]

        self.order.drop(self.order[self.order["Zadanie"].isin(child_2)].index, inplace=True)
        other.order.drop(other.order[other.order["Zadanie"].isin(child_1)].index, inplace=True)

        for i, se_zadanie, ot_zadanie in zip(
            list(range(left_b)) + list(range(right_b, l)), 
            self.order["Zadanie"], 
            other.order["Zadanie"]
        ):
            child_1[i] = ot_zadanie
            child_2[i] = se_zadanie

        child_1 = [x-1 for x in child_1]
        child_2 = [x-1 for x in child_2]

        child_1 = df.iloc[child_1].reset_index()
        child_2 = df.iloc[child_2].reset_index()
        
        child_1 = Individual(child_1)
        child_2 = Individual(child_2)
        
        return child_1, child_2

    # todo
    def __crossing_type_2(self, other):
        """
        Operator losowy z uzupełnianiem
            Krótki opis.
        """
        pass
    
        

In [4]:
from random import sample 

class Population:
    def __init__(self, starting_pool, size = 20):
        self.starting_pool = starting_pool # dane wejściowe
        self.size = size # ilosc osobników w populacji
        self.list_of_individuals = self.__gen_individuals() # list osobników
        self.mating_list = [] # lista par osobników do krzyżowania
    
    def perform_selection(self, selection_type = 1):
        """
        Selekcja. Przekierowuje do odpowiedniej funkcji selekcji.
            Opis czym jest selekcja.
            Tutaj towrzymy listę rodziców.
        """
        if selection_type == 1:
            self.mating_list = self.__selection_type_1()
        elif selection_type == 2:
            self.mating_list = self.__selection_type_2()
            
    def __selection_type_1(self):
        """
        Ranking dopasowania.
        Dobiera w pary osobniki o najlepszej wartości. (Najmniejszej)
        """
        self.list_of_individuals.sort(key=lambda x : x.value)
        return list(zip(
            self.list_of_individuals[::2], 
            self.list_of_individuals[1::2]
        ))
        
    def __selection_type_2(self):
        """
        Selekcja na zasadzie turnieju
        """
        pairs = []
        
        for i in range(len(self.list_of_individuals)//2):
            fighters = sample(self.list_of_individuals, 4)

            if fighters[0].value >= fighters[1].value:
                winner_1 = fighters[0]
            else:
                winner_1 = fighters[1]

            if fighters[2].value >= fighters[3].value:
                winner_2 = fighters[2]
            else:
                winner_2 = fighters[3]

            pairs.append((winner_1, winner_2))
        return pairs
    
    def perform_crossing(self, crossing_type = 1):
        """
        Krzyżowanie. Przekierowuje do funckji z odpowiednim typem krzyżowania.
            Tutaj tworzymy nową populacje
        """
        children = []
        for male, female in self.mating_list:
            child_1, child_2 = male.cross(female, cross_type = crossing_type)
            children.append(child_1)
            children.append(child_2)
        
        self.__update_population(children)
        
    def __update_population(self, children):
        self.list_of_individuals = self.list_of_individuals + children
        self.list_of_individuals.sort(key=lambda x: x.value)
        self.list_of_individuals = self.list_of_individuals[:self.size]
        
    
    def show_mating_pairs(self):
        for male, female in self.mating_list:
            print(f"Male: {male.value} & Female: {female.value}")
    
    def __gen_individuals(self):
        """
        Tworzenie osobników na podstawie danych wejściowych.
        Nic nie zwraca. Wypełnia list_of_individuas.
        """
        return [Individual(self.starting_pool.sample(frac=1).reset_index(drop=True)) for x in range(self.size)]
            
    def show_individuals(self):
        """
        Zwraca reprezentację wszystkich osobników w populacji w kolejności
        od najlepszego do najgorszego.
        """
        sorted_list = sorted(self.list_of_individuals, key=lambda x: x.value)
        for indiv in sorted_list:
            print(repr(indiv))

    def get_best_individual(self):
        """
        Zwraca najlepszego osobnika w populacji.
        """
        return sorted(self.list_of_individuals, key=lambda x: x.value)[0]
        
        
        

In [ ]:
population = Population(df, size=6)

population.perform_selection()
population.show_mating_pairs()
population.perform_crossing()
population.show_individuals()

# population.perform_selection(selection_type=2)
# population.show_mating_pairs()
# population.perform_crossing(crossing_type=2)
# population.show_individuals()

In [ ]:
l = 50
left_b, right_b = np.random.randint(10, l//2), np.random.randint(l//2, l - 10)

child_1 = [None for i in range(l)]
child_2 = [None for i in range(l)]

for i in range(left_b, right_b):
    child_1[i] = se.order["Zadanie"].iloc[i]
    child_2[i] = ot.order["Zadanie"].iloc[i]

se.order.drop(se.order[se.order["Zadanie"].isin(child_2)].index, inplace=True)
ot.order.drop(ot.order[ot.order["Zadanie"].isin(child_1)].index, inplace=True)

for i, se_zadanie, ot_zadanie in zip(list(range(left_b)) + list(range(right_b, l)), se.order["Zadanie"], ot.order["Zadanie"]):
    child_1[i] = ot_zadanie
    child_2[i] = se_zadanie
    
child_1 = [x-1 for x in child_1]
child_2 = [x-1 for x in child_2]

child_1 = df.iloc[child_1]
child_2 = df.iloc[child_2]

In [ ]:
nw = df.iloc[child_1]

In [ ]:
nw = nw.reset_index()

In [ ]:
nw